## Credit

The following code was adapted heavily from the following example:
https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

## Imports and Setup

#### Run this if you are working in Colab

In [ ]:
# Run this one if you are working in Colab
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

display = Display(visible=0, size=(400,300))
display.start()

In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from collections import namedtuple
from itertools import count

from PIL import Image

from ipywidgets import IntProgress
from IPython.display import display
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Replay Buffer

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## Model Architecture

In [ ]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

## Class for storing an agent

In [ ]:
class Agent():
    def __init__(self, env=None):
        self.policy_net = DQN(screen_height, screen_width, n_actions).to(device)
        self.target_net = DQN(screen_height, screen_width, n_actions).to(device)
        self.optimizer = optim.RMSprop(self.policy_net.parameters())
        self.steps_done = 0
        self.episode_durations = []
        self.env = gym.make('CartPole-v0').unwrapped
        self.current_screen = None
        self.state = None


## Screen Processing

In [ ]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])

def get_cart_location(screen_width, env):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

def get_screen(env):
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    cart_location = get_cart_location(screen_width, env)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0).to(device)

## More Setup

In [ ]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
env = gym.make('CartPole-v0').unwrapped
env.reset()
init_screen = get_screen(env)
_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.n
env.close()

def select_action(state, agent, policy_net_parent=None):
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * agent.steps_done / EPS_DECAY)
    agent.steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return agent.policy_net(state).max(1)[1].view(1, 1)
    else:
        if policy_net_parent and random.random() < 0.5:
            return policy_net_parent(state).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

## Optimize Model Function

In [ ]:
def optimize_model(agent, memory):
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = agent.policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = agent.target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    agent.optimizer.zero_grad()
    loss.backward()
    for param in agent.policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    agent.optimizer.step()

## Run Our NEW-PBT Training

In [ ]:
BATCH_SIZE = 128

GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

NUM_EPS = 1000
TARGET_UPDATE = 10
SCORED_EPS = min(100, round(0.2 * NUM_EPS))

POP_SIZE = 5
NUM_GENS = 3

In [ ]:
t0 = time.time()
progress_bar = IntProgress(min=0, max=NUM_GENS * POP_SIZE * NUM_EPS) # instantiate the bar
display(progress_bar) # display the bar

best_pnet = None

for gen in range(NUM_GENS):
    
    print(f"Generation: {gen + 1}/{NUM_GENS}")
    cur_best_pnet = None
    best_score = 0
    
    for ind in range(POP_SIZE):
        
        print(f"\tIndividual: {ind + 1}/{POP_SIZE}")

        agent = Agent()
        memory = ReplayMemory(10000)

        for i_episode in range(NUM_EPS):
            # Initialize the environment and state
            agent.env.reset()
            last_screen = get_screen(agent.env)
            current_screen = get_screen(agent.env)
            state = current_screen - last_screen
            for t in count():
                # Select and perform an action
                action = select_action(state, agent, best_pnet)
                _, reward, done, _ = agent.env.step(action.item())
                reward = torch.tensor([reward], device=device)

                # Observe new state
                last_screen = current_screen
                current_screen = get_screen(agent.env)
                if not done:
                    next_state = current_screen - last_screen
                else:
                    next_state = None

                # Store the transition in memory
                memory.push(state, action, next_state, reward)

                # Move to the next state
                state = next_state

                # Perform one step of the optimization (on the target network)
                optimize_model(agent, memory)
                if done:
                    agent.episode_durations.append(t + 1)
                    break

            # Update the target network, copying all weights and biases in DQN
            if i_episode % TARGET_UPDATE == 0:
                agent.target_net.load_state_dict(agent.policy_net.state_dict())
              
            progress_bar.value += 1
                
        # Calculate average score for agent over its lifetime
        score = sum(agent.episode_durations[-SCORED_EPS:]) / len(agent.episode_durations[-SCORED_EPS:])
        print(f"\t\tScore: {score}")
        
        # If the agent was better than the current best agent, then update the best agent
        if score > best_score:
            cur_best_pnet = agent.policy_net
            best_score = score
        
        agent.env.render()
        agent.env.close()
    
    # Set the best parent agent to be the best agent from the current generation
    best_pnet = cur_best_pnet

print('Complete')
print('Time Elapsed', time.time() - t0)

IntProgress(value=0, max=15000)

Generation: 1/3
	Individual: 1/5
		Score: 41.4
	Individual: 2/5
		Score: 48.16
	Individual: 3/5
		Score: 25.34
	Individual: 4/5
		Score: 43.09
	Individual: 5/5
		Score: 69.45
Generation: 2/3
	Individual: 1/5
		Score: 45.11
	Individual: 2/5
		Score: 42.92
	Individual: 3/5
		Score: 39.65
	Individual: 4/5
		Score: 40.19
	Individual: 5/5
		Score: 43.84
Generation: 3/3
	Individual: 1/5
		Score: 39.54
	Individual: 2/5
		Score: 46.22
	Individual: 3/5
		Score: 34.65
	Individual: 4/5
		Score: 45.71
	Individual: 5/5
		Score: 44.85
Complete
Time Elapsed 8261.845029115677


In [ ]:
best_score

46.22

## Run Baseline OLD PBT Approach

In [ ]:
BATCH_SIZE = 128

GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

NUM_EPS = 10
TARGET_UPDATE = 10
SCORED_EPS = min(100, round(0.2 * NUM_EPS))

POP_SIZE = 2
NUM_GENS = 2

In [ ]:
t0 = time.time()
progress_bar = IntProgress(min=0, max=NUM_EPS) # instantiate the bar
display(progress_bar) # display the bar

best_score = 0
    
agents = [Agent() for _ in range(POP_SIZE * NUM_GENS)]
memory = ReplayMemory(10000)
    
for i_episode in range(NUM_EPS):
    for agent in agents:
        # Initialize the environment and state
        agent.env.reset()
        last_screen = get_screen(agent.env)
        agent.current_screen = get_screen(agent.env)
        agent.state = agent.current_screen - last_screen
    
    running = [1] * POP_SIZE * NUM_GENS
    t = 0

    while sum(running) > 1:
        for j, agent in enumerate(agents):
            if running[j]:
                # Select and perform an action
                action = select_action(agent.state, agent)
                _, reward, done, _ = agent.env.step(action.item())
                reward = torch.tensor([reward], device=device)

                # Observe new state
                last_screen = agent.current_screen
                agent.current_screen = get_screen(agent.env)
                if not done:
                    next_state = agent.current_screen - last_screen
                else:
                    next_state = None

                # Store the transition in memory
                memory.push(agent.state, action, next_state, reward)

                # Move to the next state
                agent.state = next_state

                if done:
                    agent.episode_durations.append(t + 1)
                    running[j] = 0
                    break
        
        for agent in agents:
            # Perform one step of the optimization (on the target network)
            optimize_model(agent, memory) 
        
        t += 1
        
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        for agent in agents:
            agent.target_net.load_state_dict(agent.policy_net.state_dict())
    
    progress_bar.value += 1

# Calculate average score for agent over its lifetime
for agent in agents:
    score = sum(agent.episode_durations[-SCORED_EPS:]) / len(agent.episode_durations[-SCORED_EPS:])
    best_score = max(best_score, score)     

print('Complete')
print('Time Elapsed', time.time() - t0)

In [ ]:
best_score